In [ ]:
%pip install langchain

In [7]:
# from torch import cuda, bfloat16
# import torch
# import transformers
# from transformers import AutoTokenizer
# from time import time
# import chromadb
# from chromadb.config import Settings
import os

from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores.pgvector import PGVector

In [2]:
from langchain.llms import HuggingFaceTextGenInference

llm = HuggingFaceTextGenInference(
    inference_server_url="http://localhost:8080/",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
)
llm("[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  \
        Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \
        Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, \
        or is not factually coherent, explain why instead of answering something not correct. \n<</SYS>>\n \
        How to deploy a container on K8s?[/INST]")

" Hello! I'd be happy to help you with deploying a container on Kubernetes (K8s). Here are the general steps you can follow:\n\n1. Create a Kubernetes cluster: You can create a Kubernetes cluster on your local machine using tools like Minikube or Kind. You can also use a cloud provider like Google Kubernetes Engine (GKE), Amazon Elastic Container Service for Kubernetes (EKS), or Microsoft Azure Kubernetes Service (AKS).\n2. Prepare your container image: You'll need to create a container image that includes your application and any dependencies it requires. You can use a tool like Docker to create the image.\n3. Create a Kubernetes deployment: Once you have your container image ready, you can create a Kubernetes deployment to manage the lifecycle of your containers. You can use the `kubectl create deployment` command to create a deployment.\n4. Expose the deployment as a service: To make your application accessible from outside the cluster, you'll need to expose it as a Kubernetes servi

In [3]:
loader = TextLoader("sotu.txt", encoding="utf8")
documents = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [5]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [8]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "5432")),
    database=os.environ.get("PGVECTOR_DATABASE", "postgres"),
    user=os.environ.get("PGVECTOR_USER", "postgres"),
    password=os.environ.get("PGVECTOR_PASSWORD", "secretpassword"),
)

In [11]:
COLLECTION_NAME = "state_of_the_union_test"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [33]:
from langchain.prompts import PromptTemplate
prompt_template = """[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
        Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
        Use the following pieces of context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
        
        {context}

        \n<</SYS>>\n
        Question: {question}
        Answer:[/INST]"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [32]:
retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

In [27]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    # time_1 = time()
    result = qa.run(query)
    # time_2 = time()
    # print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [28]:
query = "What were the main topics in the State of the Union in 2021? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What were the main topics in the State of the Union in 2021? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...

> Finished chain.

Result:   Sure, I'd be happy to help! Based on the text you provided, the main topics in the State of the Union in 2021 were:

1. The strength of the American people, with the President emphasizing that the country is stronger today than it was a year ago and will continue to grow stronger in the future.
2. Meeting and overcoming the challenges of our time, including protecting freedom and liberty, expanding fairness and opportunity, and saving democracy.
3. The importance of unity and coming together as one people to face the challenges ahead.
4. The potential for a bright future, with the President expressing optimism about the country's ability to turn every crisis into an opportunity and to achieve great things.

Overall, the President's message emphasized the resilience and strength of the American people, the need for u

In [29]:
query = "What is the nation economic status? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What is the nation economic status? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...

> Finished chain.

Result:   Based on the State of the Union address, the nation's economic status is strong and resilient. The President highlighted the country's ability to turn every crisis into an opportunity, citing the nation's history of overcoming challenges and emerging stronger. The address emphasized the importance of protecting freedom and liberty, expanding fairness and opportunity, and saving democracy. The President expressed optimism about the nation's future, citing the strength and resilience of the American people. The address did not provide a detailed summary of the nation's economic status, but it emphasized the importance of meeting the challenges of the present moment and working together as one people to build a better future.


In [30]:
docs = db.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: What is the nation economic status? Summarize. Keep it under 200 words.
Retrieved documents: 4
Source:  ./sotu.txt
Text:  And built the strongest, freest, and most prosperous nation the world has ever known. 

Now is the hour. 

Our moment of responsibility. 

Our test of resolve and conscience, of history itself. 

It is in this moment that our character is formed. Our purpose is found. Our future is forged. 

Well I know this nation.  

We will meet the test. 

To protect freedom and liberty, to expand fairness and opportunity. 

We will save democracy. 

As hard as these times have been, I am more optimistic about America today than I have been my whole life. 

Because I see the future that is within our grasp. 

Because I know there is simply nothing beyond our capacity. 

We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. 

The only nation that can be defined by a single word: possibilities. 

So on this night, in our 245t